# SICP-3.2.4 Digital circuit simulator

## Event driven simulation

### helper functions to validate signals

In [4]:
(define (logical-not s) 
  (cond ((= s 0) 1) 
        ((= s 1) 0) 
        (else (error "Invalid signal" s))))

(define (logical-and s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (and s1 s2))))    

(define (logical-or s1 s2)
  (cond((not(or (= s1 0) (= s1 1))) error "Invalid signal" s1 )
       ((not(or (= s2 0) (= s2 1))) error "Invalid signal" s2 )
        (else (or s1 s2))))    

; define proc to call all procs in a given list
(define (call-each procedures)
  (if (null? procedures) 
    'done 
    (begin ((car procedures)) 
            (call-each (cdr procedures)))))


call-each

In [5]:
; wire object holds a signal
; when signal changed - all actions get called

(define (make-wire) 
  (let ((signal-value 0) (action-procedures '()))  ; init to 0 with empty actions list
    (define (set-my-signal! new-value) 
      (if (not (= signal-value new-value)) 
          (begin (set! signal-value new-value) 
                 (call-each action-procedures)) 
          'done)) 
    (define (accept-action-procedure! proc)        ; add action to list
      (set! action-procedures (cons proc action-procedures)) 
      (proc)) 
    (define (dispatch m) 
      (cond ((eq? m 'get-signal) signal-value) 
            ((eq? m 'set-signal!) set-my-signal!) 
            ((eq? m 'add-action!) accept-action-procedure!) 
            (else (error "Unknown operation: WIRE" m)))) 
    dispatch))


make-wire

In [11]:
; create 6 wires for implementing half-adder
(define a (make-wire)) 
(define b (make-wire)) 
(define c (make-wire)) 
(define d (make-wire)) 
(define e (make-wire)) 
(define s (make-wire))


s

In [14]:
; syntactic sugar. call object method as a function call [ie: obj.do(); == do(obj); ]
(define (get-signal wire) (wire 'get-signal)) 
(define (set-signal! wire new-value) ((wire 'set-signal!) new-value)) 
(define (add-action! wire action-procedure) ((wire 'add-action!) action-procedure))

add-action!

![title](img/logic-gates.png)

In [25]:
; inverter accepts 2 wires, and creates invert-input function
; that been added to the wire actions-list (connect to the wire)
(define (inverter input output) 
  (define (invert-input) 
    (let ((new-value (logical-not (get-signal input))))
      (after-delay inverter-delay 
        (lambda () 
          (set-signal! output new-value))))) 
  (add-action! input invert-input) 'ok) 

; and-gate accepts 3 wires, create action and bind to both input wires
(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! a1 and-action-procedure)
  (add-action! a1 and-action-procedure)
  'ok)

; or-gate accepts 3 wires, create action and bind to both input wires
(define (or-gate o1 o2 output)
  (define (or-action-procedure)
    (let ((new-value (logical-or (get-signal o1) (get-signal o2))))
      (after-delay or-gate-delay 
        (lambda () 
          (set-signal! output new-value)))))
  (add-action! o1 or-action-procedure)
  (add-action! o2 or-action-procedure)
  'ok)


or-gate

![title](img/half-adder.png)

In [26]:
; construction of gates, by providing wires as args
(or-gate a b d)
(and-gate a b c)
(inverter c e)
(and-gate d e s)


unbound-variable: Unbound variable: or-gate-delay

In [27]:
(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s) 'ok))

half-adder

![title](img/full-adder.png)

In [28]:
(define (full-adder a b c-in sum c-out) 
  (let ((s (make-wire)) (c1 (make-wire)) (c2 (make-wire))) 
    (half-adder b c-in s c1) 
    (half-adder a s sum c2) 
    (or-gate c1 c2 c-out) 'ok))


full-adder